--- Day 4: Passport Processing ---
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

Your puzzle answer was 206.

The first half of this puzzle is complete! It provides one gold star: *

In [12]:
import pandas as pd
import numpy as np

In [2]:
#ingest data between blank lines as associate with a single passport, i.e. new line != new passport.
#so can't go straight to df, use a detour to manage split by blank line

with open('C:/Users\Loren/Desktop/advent_of_code_day4.txt') as f:
    lines = f.read()

tests = lines.split("\n\n") 

In [3]:
df = pd.DataFrame(tests)

In [4]:
df = df.rename(columns={0:'raw'})

In [5]:
#check that data is grouped per passport - inspection check - YES!
df.head()

,raw
0,ecl:#eef340 eyr:2023 hcl:#c0946f pid:244684338...
1,pid:303807545 cid:213 ecl:gry hcl:#fffffd\neyr...
2,hcl:#c0946f byr:1933 eyr:2025 pid:517067213 hg...
3,pid:5253256652 byr:2009 hgt:152cm iyr:1989 eyr...
4,iyr:2013\npid:862607211 eyr:2020\nhgt:174cm\nb...


In [6]:
#make each entry a list of fields and values
def make_list(raw):
    raw1 = str(raw) #convert to string so can use string split into list on spaces
    raw2 = raw1.split()
    return raw2
    
df['passport_fields'] = df.apply(lambda x: make_list(x['raw']), axis=1)

In [7]:
#validation rules 
# if 8 keys present with value - valid, if 7 but only missing cid - valid, otherwise invalid. 

def valid_doc(passport_fields):
    fields_count = len(passport_fields) #for assessing validity
    
    cid_included = 0 #for assessing validity with cid exception
    
    for field in passport_fields:
        if field[:3] == 'cid':
            cid_included += 1 #cid_included == 1 means cid is in list i.e. 7 with cid_included == 1 is invalid because something else is missing. 

    #validity test
    if fields_count == 8: #all 8 fields = valid
        return 1 #valid
    elif (fields_count == 7) & (cid_included == 0): #all fields but cid i.e. North Pole Creds = valid
        return 1 #valid 
    else:
        return 0 #invalid
    
df['valid_doc'] = df.apply(lambda x: valid_doc(x['passport_fields']), axis=1)    

In [8]:
df.head(20)

,raw,passport_fields,valid_doc
0,ecl:#eef340 eyr:2023 hcl:#c0946f pid:244684338...,"[ecl:#eef340, eyr:2023, hcl:#c0946f, pid:24468...",1
1,pid:303807545 cid:213 ecl:gry hcl:#fffffd\neyr...,"[pid:303807545, cid:213, ecl:gry, hcl:#fffffd,...",1
2,hcl:#c0946f byr:1933 eyr:2025 pid:517067213 hg...,"[hcl:#c0946f, byr:1933, eyr:2025, pid:51706721...",1
3,pid:5253256652 byr:2009 hgt:152cm iyr:1989 eyr...,"[pid:5253256652, byr:2009, hgt:152cm, iyr:1989...",1
4,iyr:2013\npid:862607211 eyr:2020\nhgt:174cm\nb...,"[iyr:2013, pid:862607211, eyr:2020, hgt:174cm,...",1
5,ecl:grn byr:1955\npid:608098408 eyr:2027 iyr:2...,"[ecl:grn, byr:1955, pid:608098408, eyr:2027, i...",1
6,byr:1998 ecl:hzl eyr:2021\nhcl:#866857 hgt:189...,"[byr:1998, ecl:hzl, eyr:2021, hcl:#866857, hgt...",1
7,pid:075417314 hcl:#b6652a byr:1957\niyr:2011\n...,"[pid:075417314, hcl:#b6652a, byr:1957, iyr:201...",1
8,iyr:1993 hgt:74cm eyr:1960\nbyr:2029\nhcl:2932...,"[iyr:1993, hgt:74cm, eyr:1960, byr:2029, hcl:2...",1
9,hgt:153cm hcl:#6b5442 pid:065240897 byr:1932\n...,"[hgt:153cm, hcl:#6b5442, pid:065240897, byr:19...",1


In [9]:
#count of valids =  answer
df['valid_doc'].sum()

206

Part 2

--- Part Two ---
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
Here are some invalid passports:

eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
Here are some valid passports:

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

Your puzzle answer was 123.

Both parts of this puzzle are complete! They provide two gold stars: **

In [10]:
#increased validation rules in addition to part 1

def valid_doc_stricter(valid_doc_part_1, passport_fields):  
    #check if valid in part 1, if not fail early
    if valid_doc_part_1 == 0:
        return 0 #invalid even on less strict rules
    else: #i.e.  valid in part 1 now check stricter rules
        stricter_score = 0 #add 1 per validation passed, min 7 to pass overall
        
        for field in passport_fields:   
            field_key = field[:3]
            field_value = field.split(':')[1] #need field value i.e. portion after the ':'

            if field_key == 'byr':
                if (len(field_value) == 4) & (int(field_value) >= 1920) & (int(field_value) <= 2002):
                    stricter_score += 1 #valid for this field
            if field_key == 'iyr':
                if (len(field_value) == 4) & (int(field_value) >= 2010) & (int(field_value) <= 2020):
                    stricter_score += 1 #valid for this field
            if field_key == 'eyr':
                if (len(field_value) == 4) & (int(field_value) >= 2020) & (int(field_value) <= 2030):
                    stricter_score += 1 #valid for this field
            if field_key == 'hgt':
                if 'cm' in field_value:
                    field_value = field_value.replace('cm', '')
                    if (int(field_value) >= 150) & (int(field_value) <= 193):
                        stricter_score += 1 #valid for this field
                elif 'in' in field_value:
                    field_value = field_value.replace('in', '')
                    if (int(field_value) >= 59) & (int(field_value) <= 76):
                        stricter_score += 1 #valid for this field
            if field_key == 'hcl':
                hcl = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
                if (field_value[:1] == '#') & (len(field_value) == 7):
                    hcl_count = 0 #need 6 to validate all chars
                    for i in field_value[1:]:
                        if i in hcl:
                            hcl_count += 1
                        if hcl_count == 6:
                            stricter_score += 1 #valid for this field
            if field_key == 'ecl':
                ecl = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']
                if (field_value in ecl):
                    stricter_score += 1 #valid for this field
            if field_key == 'pid':
                if (len(field_value) == 9) & (field_value.isnumeric()):
                    stricter_score += 1 #valid for this field
        return stricter_score
 
df['valid_doc_stricter_check_count'] = df.apply(lambda x: valid_doc_stricter(x['valid_doc'], x['passport_fields']), axis=1)    

In [13]:
df['valid_doc_stricter'] = np.where(df['valid_doc_stricter_check_count'] >= 7, 1, 0)

In [14]:
df.head(20)

,raw,passport_fields,valid_doc,valid_doc_stricter_check_count,valid_doc_stricter
0,ecl:#eef340 eyr:2023 hcl:#c0946f pid:244684338...,"[ecl:#eef340, eyr:2023, hcl:#c0946f, pid:24468...",1,6,0
1,pid:303807545 cid:213 ecl:gry hcl:#fffffd\neyr...,"[pid:303807545, cid:213, ecl:gry, hcl:#fffffd,...",1,6,0
2,hcl:#c0946f byr:1933 eyr:2025 pid:517067213 hg...,"[hcl:#c0946f, byr:1933, eyr:2025, pid:51706721...",1,7,1
3,pid:5253256652 byr:2009 hgt:152cm iyr:1989 eyr...,"[pid:5253256652, byr:2009, hgt:152cm, iyr:1989...",1,2,0
4,iyr:2013\npid:862607211 eyr:2020\nhgt:174cm\nb...,"[iyr:2013, pid:862607211, eyr:2020, hgt:174cm,...",1,7,1
5,ecl:grn byr:1955\npid:608098408 eyr:2027 iyr:2...,"[ecl:grn, byr:1955, pid:608098408, eyr:2027, i...",1,7,1
6,byr:1998 ecl:hzl eyr:2021\nhcl:#866857 hgt:189...,"[byr:1998, ecl:hzl, eyr:2021, hcl:#866857, hgt...",1,6,0
7,pid:075417314 hcl:#b6652a byr:1957\niyr:2011\n...,"[pid:075417314, hcl:#b6652a, byr:1957, iyr:201...",1,7,1
8,iyr:1993 hgt:74cm eyr:1960\nbyr:2029\nhcl:2932...,"[iyr:1993, hgt:74cm, eyr:1960, byr:2029, hcl:2...",1,0,0
9,hgt:153cm hcl:#6b5442 pid:065240897 byr:1932\n...,"[hgt:153cm, hcl:#6b5442, pid:065240897, byr:19...",1,7,1


In [15]:
df['valid_doc_stricter'].sum()

123